# Project Team Members:
   **Andres Baez, acbaez@ucsd.edu, A92112700**
   
   **Catherine Kim, cyk020@ucsd.edu, A12598619**
   
   **Eric Richards, eorichar@ucsd.edu, A12724500**
   
   **Ronald Tun, Rtun@ucsd.edu, A12458764 **
   
   **Janica Kann, jkann@ucsd.edu, A13734650**
   
   **Myung Sun Kim, msk024@ucsd.edu, A12870165 **

# 0. Imports That Allow Our Project to Work: 

In [1]:
#import necessary packages
import json
import pandas as pd
import numpy as np
from textblob import TextBlob
import time

# 1. Introduction and Background:

From personal experience using Yelp, we have noticed that users tend to leave more opinionated reviews as they gain more experience reviewing restaurants.  Through this observation, we became curious if users leave “critical” reviews as time passes. In this context, we define the word “critical” as users leaving more reviews with a negative connotation.

**Research Question:** Do Yelp users become more critical in their reviews after becoming “Elite” members?

**Hypothesis:** After becoming Elite members, users will become more critical in their Yelp reviews compared to their earlier reviews. The Elite status can be used as an indicator of whether a user will become more likely to leave a critical review. 


# 2. Methods:

Within Yelp’s academic dataset are 6 .json files, we will be using 2 (review.json, user.json) files for this project. They contain full review text data including the user_id that wrote the review and the business_id the review is written for, as well as user data such as the user's friend mapping and associated metadata.
We combined user.json and review.json by matching the 22 character unique user id for each review in review.json to the same 22 character user id in user.json. This will match a review (a text string) to the corresponding user profile data (including whether they are an elite user) available in user.json.

We cleaned this, this, and this from this dataset. We utilized this, this, and this from this dataset. 
We’re looking to see if there is a relationship between the time a user becomes an elite member to how they leave more critical reviews after this status change. We’ll be comparing how the user left reviews prior to becoming an elite member to how the user leaves reviews after the user becomes an Elite member to see if there is a difference.


In [2]:
#Read in both data frames
#Set filename and read through review JSON file line by line
if False:
    filename_review = "/Users/andresbaez/Desktop/COGS108_Project/yelp_dataset/review.json"
    #Create dataframe that will contain review data
    df_review = pd.DataFrame()
    #go through each JSON object and create a dictionary out of its key-value pairs
    st = time.time()
    with open(filename_review,'r') as f:
        for line in f:
            data = json.loads(line)
            #append each JSON object to df_review
            df_review = df_review.append(data, ignore_index=True)       
            if (time.time() - st) > 60:
                break
    df_review.to_csv("/Users/andresbaez/Desktop/COGS108_Project/yelp_dataset/review.csv")
    #Set filename and read through user JSON file line by line
    filename_users = "/Users/andresbaez/Desktop/COGS108_Project/yelp_dataset/user.json"
    #Create dataframe that will contain user data
    df_users = pd.DataFrame()
    #go through each JSON object and create a dictionary out of its key-value pairs
    st = time.time()
    with open(filename_users,'r') as f:
        for line in f:
            data = json.loads(line)
            #append each JSON object to df_users
            df_users = df_users.append(data, ignore_index=True)
            if (time.time() - st) > 60:
                break
    df_users.to_csv("/Users/andresbaez/Desktop/COGS108_Project/yelp_dataset/user.csv")

**PLACEHOLDER -- Dropping **

In [3]:
#Read newly exported CSV files into dataframe
df_review = pd.read_csv('./yelp_dataset/review2.csv')
df_user = pd.read_csv('./yelp_dataset/user.csv')

#Start out by dropping columns that are not necessary to our analysis
df_review.drop(['business_id','cool','funny','useful'],axis=1,inplace=True)
df_user.drop(['friends','compliment_cool','compliment_cute','compliment_funny','compliment_hot','compliment_list',
              'compliment_more','compliment_note','compliment_photos','compliment_writer','compliment_plain','compliment_profile',
              'cool','fans','funny','name','useful','yelping_since'],axis=1,inplace=True)

**PLACEHOLDER -- Writing Functions**

In [4]:
#Want to parse out year from date (YEAR-DAY-MONTH)
def getYear(date):
    year = date.split("-")[0]
    return int(year)

#assumes stars is int and reviewText is string
def find_sentiment(stars, reviewText):
    review = TextBlob(reviewText)
    
    #convert review sentiment from [-1,1] to [0,5]
    reviewScore = (review.sentiment[0] + 1) / 2
    reviewScore = reviewScore * 5
    
    #return average of scores
    score = (stars + reviewScore) / 2
    return score
    
def isElite(year, elite):
    if year in elite:
        return True
    else:
        return False
    
def convert_string_to_list(elite):
    elite = str(elite)
    if elite == "NaN" or elite == 'nan':
        return
    elite.strip()
    elite_list = elite.split(',')    
    out = []
    for item in elite_list:
        out.append(int(item))
    return list(out)

**PLACEHOLDER -- Averaging sentiment of "elite" and non "elite" members**

In [5]:
#Want to convert string values to int before dropping nulls
df_user['elite'] = df_user['elite'].apply(convert_string_to_list)

#Want to drop any columns where the user has never been an elite member
df_user.dropna(subset =['elite'],inplace=True)

#Apply function to obtain year from date in both dataframes
df_review['date'] = df_review['date'].apply(getYear)

#Want to merge our user and review data now that we've dropped unnecessary rows and columns
df = pd.merge(df_user,df_review,on='user_id',how='right')


TypeError: ("'Series' objects are mutable, thus they cannot be hashed", 'occurred at index 0')

In [7]:
#Apply function to find if user was an Elite member at the time of the review
#DO THIS IS IN A LOOP INSTEAD
for row, index in df.iterrows():
    row['Elite_Reviewer'] = isElite(row['date'],row['elite'])
    
#df['Elite_Reviewer'] = df.apply(lambda x: isElite(df['date'], df['elite']), axis=1)

#Apply function to find sentiment for each review in df_review
#df_review['critical_index'] = df_review.apply(lambda x: find_sentiment(df_review['stars'], df_review['text'][0]), axis=1)

TypeError: 'int' object is not subscriptable

**PLACEHOLDER -- Statistical Analysis**

# 4. Conclusions:

# 5. Future Work:

# 6. Resources: